# Introduction

One of the perennial issues with living in Singapore is the cost of cars. Singapore consistently ranks as one of the most expensive cities to live in and this is in large part due to the cost of our cars and our houses (food is _thankfully_ super affordable). 

For young adults who are just starting work, the common advice given is to not buy a car. Looking at [this](https://blog.seedly.sg/taking-grab-vs-own-car-which-is-cheaper/) comparison by seedly, it appears that one could potentially save upwards of $100,000 by taking a cab everyday instead of owning a car. But... what if you *really* need one? Maybe you work at a super _ulu_ (Singlish slang for far-out/abandoned/hard to access) place and its really hard to get a Grab/Uber. 

So, you decide to buy a car! 

In [1]:
import pandas as pd
import plotly.express as px 
import numpy as np
import plotly.io as pio
import matplotlib.pyplot as plt

In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go 

In [3]:
# set various parameters
car_price = 65000
# start_cap = 140000
maintenance_monthly = 500*12*10

In [4]:
# create range of downpayments possible
downpayment_list = [i for i in range(int(car_price*0.3),car_price,200)]

In [5]:
# create dataframe
car_loan_df = pd.DataFrame({"Downpayment" : downpayment_list
                            })

# add total cost of car to dataframe
for z in range(1,8):
    car_loan_df["{} Year".format(z)] = [((0.0298*z*(car_price - downpayment_list[i])+(car_price - downpayment_list[i])))+downpayment_list[i]+maintenance_monthly for i in range(0,len(downpayment_list))]

for z in range(1,8):
    car_loan_df["{} Year(M)".format(z)] = [(((0.0298*z*(car_price - downpayment_list[i])+(car_price - downpayment_list[i])))/(z*12))+(maintenance_monthly/12/10) for i in range(0,len(downpayment_list))]

# melt for easier plotting
car_melt = pd.melt(car_loan_df, id_vars="Downpayment")

In [34]:
fig = make_subplots(rows=2, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.15,
                    subplot_titles=("Total Cost","Monthly Cost"))

for t in range(1,8):
    fig.add_trace(go.Scatter(x=car_loan_df["Downpayment"],y=car_loan_df["{} Year".format(t)], mode='lines',name='{} Year'.format(t), hovertemplate=
    '<b>Downpayment</b>: $%{x}<br>'+
    '<b>Total Cost</b>: $%{y}'), row=1, col=1)

for t in range(1,8):
    fig.add_trace(go.Scatter(x=car_loan_df["Downpayment"],y=car_loan_df["{} Year(M)".format(t)], mode='lines',name='{} Year'.format(t), hovertemplate=
    '<b>Downpayment</b>: $%{x}<br>'+
    '<b>Monthly Cost</b>: $%{y}'), row=2, col=1)

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.update_layout({"plot_bgcolor":"white"})
fig.update_xaxes(title_text="Downpayment", showgrid=True, gridwidth=1, gridcolor='lightgrey', row=2,col=1)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', row=1,col=1)
fig.update_yaxes(title_text="Cost", showgrid=True, gridwidth=1, gridcolor='lightgrey', row=1,col=1)
fig.update_yaxes(title_text="Cost", showgrid=True, gridwidth=1, gridcolor='lightgrey', row=2,col=1)
fig.show()

In [35]:
fig = px.line(car_loan_df,x="Downpayment",y=["1 Year", "2 Year", "3 Year", "4 Year", "5 Year", "6 Year", "7 Year"],title="True Cost of Car with Loan Interest and Monthly Expenses", labels={"variable":"Loan Term","value":"True Cost of Car"},
                 color_discrete_sequence=px.colors.qualitative.Bold)

fig.update_layout({"plot_bgcolor":"white"})
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')

fig.show()

In [39]:
fig = px.line(car_loan_df,x="Downpayment",y=["1 Year(M)", "2 Year(M)", "3 Year(M)", "4 Year(M)", "5 Year(M)", "6 Year(M)", "7 Year(M)"],title="Monthly Cost of Car with Loan Interest and Monthly Expenses", labels={"variable":"Loan Term","value":"Monthly Payment"},
                 color_discrete_sequence=px.colors.qualitative.Bold)

fig.update_layout({"plot_bgcolor":"white"})
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.show()

In [6]:
# function to calculate compound interest given monthly contributions
def fv(monthly, interest_rate, time):
    return (monthly * ((((1 + interest_rate)**time)-1)/(interest_rate)))

# function to calculate compound interest given monthly contribution and starting amount
def ifv(initial, monthly, interest_rate, time):
    return ((initial*((1+interest_rate)**time)) + (monthly * ((((1 + interest_rate)**time)-1)/(interest_rate))))

In [7]:
# function to calculate the return on investment given 10 year capital to ivnest
def car_tradeoff_calc(car_price,downpayment,years,bank_int,capital_invest,simir):
    total_cost = ((bank_int*years*(car_price - downpayment)+(car_price - downpayment))) + downpayment + maintenance_monthly
    pay_monthly = (total_cost - downpayment)/(years*12)
    pre_invest_monthly = max(0,((capital_invest-downpayment)/(10*12)-pay_monthly))
    pre_invest_total = fv(pre_invest_monthly, simir, years*12)
    post_invest_monthly = ((capital_invest-((pay_monthly+ pre_invest_monthly)*years*12)-downpayment)/((10-years)*12))
    post_invest_total = ifv(pre_invest_total,post_invest_monthly,simir,((10-years)*12))
    total_returns = pre_invest_total + post_invest_total
    return(post_invest_total)

In [8]:
# create dataframe
payoff_df = pd.DataFrame({"Downpayment" : downpayment_list
                            })

# make sure one has enough capital to pay for the car fully
if (start_cap < car_loan_df.iloc[0,-1]):
    print("Error: Starting capital not large enough")
else:
    for z in range(1,8):
        payoff_df["{} yr".format(z)] = [car_tradeoff_calc(car_price,downpayment_list[i],z,0.0298,start_cap,0.007) for i in range(0,len(downpayment_list))]

#melt it for easier plotting
payoff_df_melt = pd.melt(payoff_df, id_vars="Downpayment")

NameError: name 'start_cap' is not defined

In [15]:
fig = px.line(payoff_df_melt,x="Downpayment",y="value",color="variable",labels={
                     "Downpayment": "Initial Downpayment",
                     "value": "Return on Investment",
                     "variable": "Loan Term"
                 }, title="Payoff Diagram, with 10 year capital: {}".format(start_cap))
fig.show()

NameError: name 'payoff_df_melt' is not defined

In [16]:
# function to calculate the return on investment given monthly budget
def car_tradeoff_calc_monthly(car_price,downpayment,years,bank_int,cap_month,simir, maintenance_monthly):
    total_cost = ((bank_int*years*(car_price - downpayment)+(car_price - downpayment))) + downpayment + maintenance_monthly
    pay_monthly = (total_cost - downpayment)/(years*12)
    if cap_month < pay_monthly:
        loss = (pay_monthly - cap_month)*years*12
        post_invest_monthly = cap_month
        post_invest_total = fv(post_invest_monthly, simir, ((10-years)*12))
        total_returns = post_invest_total - loss
        return (total_returns)
    else: 
        pre_invest_monthly = max(0,(cap_month-pay_monthly))
        pre_invest_total = fv(pre_invest_monthly, simir, years*12)
        post_invest_monthly = cap_month
        post_invest_total = ifv(pre_invest_total,post_invest_monthly,simir,((10-years)*12))
        total_returns = pre_invest_total + post_invest_total
        return(post_invest_total)

month_cap = 600

In [17]:
#create dataframe
payoffmonth_df = pd.DataFrame({"Downpayment" : downpayment_list
                            })

# add return on investments into dataframe
for z in range(1,8):
    payoffmonth_df["{} yr".format(z)] = [car_tradeoff_calc_monthly(car_price,downpayment_list[i],z,0.0298,month_cap,0.007,maintenance_monthly) for i in range(0,len(downpayment_list))]

# melt it for easier plotting
payoffmonth_df_melt = pd.melt(payoffmonth_df, id_vars="Downpayment")

In [18]:
fig = px.line(payoffmonth_df_melt,x="Downpayment",y="value",color="variable",labels={
                     "Downpayment": "Initial Downpayment",
                     "value": "Return on Investment",
                     "variable": "Loan Term"
                 }, title="Payoff Diagram, with monthly budget: {}".format(month_cap))

fig.update_layout({"plot_bgcolor":"white"})
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
fig.add_shape(type="line",
    x0=20000, y0=0, x1=downpayment_list[-1], y1=0,
    line=dict(color="red",width=1,dash='dot')
)
fig.show()